### Use full_music_data and/or the two summary data sets (with artists and years) of music characteristics, to develop measures of music similarity. Using your measure, are artists within genre more similar than artists between genres?

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import ast
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

In [5]:
artists = pd.read_csv('data_by_artist.csv')
years = pd.read_csv('data_by_year.csv')
artists = artists.drop_duplicates(subset='artist_name', keep='last')
full = pd.read_csv('full_music_data.csv')
influence = pd.read_csv('influence_data.csv')

In [6]:
#remove songs by artists that arent in the artists table
#duplicate songs w/ more than one artist
#takes my computer about a minute to run
#can ignore the warnings about setting a copy of a slice from a dataframe
allArtists = list(artists['artist_id'])
newdict = {}

full['artists_id'] = [int(x.split(',')[0][1:-1]) if len(x.split(',')) == 1 else [int(y) for y in x[1:-1].split(',')] for x in list(full['artists_id'])]
full['artist_names'] = [ast.literal_eval(x) if len(ast.literal_eval(x)) > 1 else ast.literal_eval(x)[0] for x in full['artist_names']]
floor = min(full['tempo'])
span = max(full['tempo']) - floor
full['tempo'] = [(x-floor)/span for x in full['tempo']]

last_id = 0
row = 0
for i in tqdm(range(len(full))): 
    thisRow = full.iloc[i,:]
    artIDs = thisRow['artists_id']
    artNames = thisRow['artist_names']
    
    if type(artIDs) == int:
        if artIDs == last_id: 
            thisRow['artists_id'] = artIDs
            thisRow['artist_names'] = artNames
            newdict[row] = thisRow.to_dict()
            row+=1
            last_id = artIDs
        else:
            if artIDs in allArtists:
                thisRow['artists_id'] = artIDs
                thisRow['artist_names'] = artNames
                newdict[row] = thisRow.to_dict()
                row+=1
                last_id = artIDs
                
    else:
        for j in range(len(artIDs)):
            artID = int(artIDs[j])
            artName = str(artNames[j])
            
            if artID in allArtists:
                thisRow['artists_id'] = artID
                thisRow['artist_names'] = artName
                newdict[row] = thisRow.to_dict()
                row+=1
    i+=1

full = pd.DataFrame.from_dict(newdict,'index')

artistGenres = {} #adds song genre to full
for i in tqdm(range(len(influence))):
    row = influence.iloc[i]
    influencer = row['influencer_id']
    follower = row['follower_id']
    infGenre = row['influencer_main_genre']
    folGenre = row['follower_main_genre']
    artistGenres[influencer] = infGenre
    artistGenres[follower] = folGenre

def getGenre(row):
    artist = row['artists_id']
    if type(artist) == list:
        print(row)
    try:
        genre = artistGenres[artist]
    except KeyError:
        genre = 'N/A'
    return genre

full['genre'] = full.apply(lambda row: getGenre(row), axis = 1)
full


  0%|                                                                                        | 0/98340 [00:00<?, ?it/s]C:\Users\jonat\anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\jonat\anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\jonat\anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

,artist_names,artists_id,danceability,energy,valence,tempo,loudness,mode,key,acousticness,instrumentalness,liveness,speechiness,explicit,duration_ms,popularity,year,release_date,song_title (censored),genre
0,Fat Freddy's Drop,178301,0.600,0.365,0.131,0.532777,-13.083,0,9,0.06720,0.585000,0.0921,0.0498,0,437200,54,2005,2005,Ernie,Electronic
1,Fat Freddy's Drop,178301,0.874,0.326,0.179,0.490063,-13.302,0,11,0.01360,0.148000,0.0993,0.1310,0,581008,53,2005,2005,Wandering Eye,Electronic
2,Fat Freddy's Drop,178301,0.670,0.531,0.336,0.571037,-8.267,0,9,0.01560,0.345000,0.3060,0.0377,0,431293,55,2009,8/7/2009,The Raft,Electronic
3,Alexander O'Neal,625201,0.761,0.702,0.850,0.429237,-8.523,1,7,0.10800,0.000031,0.0935,0.0389,0,304427,34,1991,1/1/1991,All True Man,R&B;
4,Alexander O'Neal,625201,0.661,0.828,0.902,0.471455,-12.673,0,11,0.27600,0.000001,0.2870,0.0390,0,264933,37,1987,7/29/1987,(What Can I Say) To Make You Love Me,R&B;
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102913,ZZ Top,690254,0.276,0.892,0.715,0.329693,-7.035,1,11,0.40900,0.000000,0.7140,0.0893,0,115973,33,1975,4/18/1975,Jailhouse Rock - **** Remaster,Pop/Rock
102914,ZZ Top,690254,0.700,0.592,0.906,0.450025,-10.434,1,11,0.10800,0.018400,0.1180,0.0681,0,263627,32,1976,11/29/1976,It's Only Love,Pop/Rock
102915,ZZ Top,690254,0.709,0.709,0.863,0.456977,-12.023,1,2,0.11800,0.000011,0.1250,0.0370,0,158400,31,1987,1987,Balinese,Pop/Rock
102916,ZZ Top,690254,0.552,0.651,0.533,0.661835,-10.624,0,4,0.00494,0.034900,0.1320,0.2290,0,232533,43,1992,4/13/1992,La Grange - **** Remaster,Pop/Rock


In [11]:
idDict = {}
for i in range(len(artists)):
    try:
        idDict[artists['artist_id'][i]] = artists['artist_name'][i]
    except KeyError:
        pass

influence = pd.read_csv('influence_data.csv')
iGraph = nx.DiGraph()
for artist in artists['artist_id']:
    iGraph.add_node(artist, weight = 0)
    
for i in range(len(influence)):
    influencer = influence["influencer_id"][i]
    follower = influence['follower_id'][i]
    
    iGraph.add_edge(influencer,follower)    

arts = []
artnames = []
scores = []
for node in iGraph.nodes():
    children = list(iGraph.successors(node))
    total = 0
    for child in children:
        total += iGraph.out_degree[child]
        
    iGraph.nodes[node]['weight'] = total
    arts.append(node)
    try:
        artnames.append(idDict[node])
    except KeyError:
        artnames.append('Unknown')
    scores.append(total)
    
   
totalInfluence = pd.DataFrame({'names':artnames, 'artists':arts, 'scores':scores})
totalInfluence = totalInfluence.sort_values(by=['scores'], ascending = False)
totalInfluence = totalInfluence.reset_index(drop=True)
totalInfluence[:10]

,names,artists,scores
0,The Beatles,754032,11237
1,Bob Dylan,66915,7928
2,The Rolling Stones,894465,6341
3,Chuck Berry,120521,5897
4,Elvis Presley,180228,4749
5,The Velvet Underground,840402,4380
6,Little Richard,824022,4371
7,Jimi Hendrix,354105,4312
8,Hank Williams,549797,4134
9,The Kinks,100160,4067


In [12]:
genreCount = full.groupby('genre').count().iloc[:,1]
genreCount = genreCount[genreCount > 1000].sort_values(ascending = False)

songStats = ['danceability', 'energy', 'valence', 'tempo', 'acousticness', 'instrumentalness', 'speechiness']        

since60 = full[full['year'] >= 1960]
genreStyles = full.groupby(['genre']).mean()
genres = list(genreCount.index)
since60 = since60[since60['genre'].isin(genres)]
since60 = since60.groupby(['year','genre']).mean()
since60.loc[1960].loc['Blues']

newDict = {}
for year in tqdm(range(1960,2021)):
    inYearRange = full[full['year'] <= year+2]
    inYearRange = inYearRange[inYearRange['year'] >= year-2]
    yearAvg = inYearRange.mean()[songStats].to_dict()
    newDict[year] = yearAvg
    for genre in genres:
        allInGenre = inYearRange[inYearRange['genre'] == genre]
        rollingAvg = allInGenre.mean()[songStats].to_dict()
        newDict[str(year)+genre] = rollingAvg
genreYears = pd.DataFrame.from_dict(newDict, orient = 'index')
genreYears

100%|██████████████████████████████████████████████████████████████████████████████████| 60/60 [00:15<00:00,  3.88it/s]


,danceability,energy,valence,tempo,acousticness,instrumentalness,speechiness
1960Pop/Rock,0.535358,0.489946,0.678651,0.477926,0.670648,0.073234,0.053703
1960R&B;,0.516806,0.449322,0.648734,0.484936,0.702474,0.026630,0.051043
1960Country,0.590282,0.417505,0.694578,0.477073,0.713350,0.052295,0.053156
1960Jazz,0.503312,0.282117,0.451191,0.455377,0.753262,0.340393,0.054833
1960Vocal,0.429970,0.248538,0.396518,0.451265,0.800083,0.013699,0.056298
...,...,...,...,...,...,...,...
2019Electronic,0.657600,0.688147,0.446659,0.473650,0.179765,0.036758,0.088609
2019International,0.478000,0.704000,0.228000,0.614128,0.016000,0.000000,0.047100
2019Folk,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019Blues,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [136]:
artistStyle = full.groupby(['artist_names']).mean()[songStats] #some values dont make sense to average but w/e
artistStyle

['Pop/Rock',
 'Electronic',
 'Reggae',
 'Jazz',
 'Country',
 'Comedy/Spoken',
 'R&B;',
 'Classical',
 'Latin',
 'Vocal',
 'Folk',
 'Easy Listening',
 'International',
 'Avant-Garde',
 'Blues',
 'Stage & Screen',
 'New Age',
 'Religious',
 "Children's",
 'Unknown']

In [137]:
genreDiff = {'baseline':np.array([])}
for i in tqdm(range(len(full))):
    row = full.iloc[i]
    try:
        genre = artistGenres[row['artists_id']]
    except KeyError:
        continue
    
    year = row['year']
    
    if genre in genres and year >= 1960:
        baseline = np.array([genreYears.loc[year].to_numpy()])
        try:
            expStats = np.array([genreYears.loc[str(year)+genre].to_numpy()])
        except KeyError:
            continue
        stats = np.array([row[songStats].to_numpy()])
        
        genreDiff['baseline'] = np.append(genreDiff['baseline'], float(cosine_similarity(stats,baseline)))
        try:
            genreDiff[genre] = np.append(genreDiff[genre],float(cosine_similarity(stats, expStats)))
        except KeyError:
            genreDiff[genre] = np.array([float(cosine_similarity(stats,expStats))])

In [88]:
for genre in list(genreDiff.keys()):
    genreDiff[genre] = np.mean(genreDiff[genre])
genreDiff

{'baseline': 0.9136464572663171,
 'Electronic': 0.9092092436173188,
 'R&B;': 0.9435223676558214,
 'Vocal': 0.946717702978509,
 'Pop/Rock': 0.9200648644288861,
 'Blues': 0.9392902750114116,
 'Jazz': 0.9085031455866949,
 'Country': 0.9458346054332495,
 'Latin': 0.9488499768315566,
 'Folk': 0.9359284078999699,
 'International': 0.9271093581940729,
 'Reggae': 0.9595173515694457,
 'Classical': 0.9321574931831886,
 'Stage & Screen': 0.8956930844550594}